In [ ]:
# imports

from llama_index import GPTListIndex, \
                        PromptHelper, \
                        LLMPredictor, \
                        ServiceContext, \
                        download_loader

from azure_openai import AzureOpenAiLLM

In [ ]:
# the config values seem to work well for davinci...

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
llm_predictor = LLMPredictor(llm=AzureOpenAiLLM(max_tokens=num_output))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

In [ ]:
# read .docx, .pdf, etc. files from 'data' subfolder and build index from the text

SimpleDirectoryReader = download_loader("SimpleDirectoryReader")

documents = SimpleDirectoryReader('./data').load_data() # type: ignore

index = GPTListIndex.from_documents(documents, service_context=service_context)

In [ ]:
# send a query to your Azure model, with document text added as part of prompt

response = index.query("YOUR CUSTOM PROMPT")

In [ ]:
print(response.response)